In [1]:
from pyspark.sql import SparkSession

# Spark session & Context
spark = (SparkSession
         .builder
         .master('local')
         .appName('kafka-streaming')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
         .getOrCreate())
sc = spark.sparkContext

KeyboardInterrupt: 

In [ ]:
# Read the message from the kafka stream
df = spark \
     .readStream \
     .format("kafka") \
     .option("kafka.bootstrap.servers", "kafka:9092") \
     .option("subscribe", "ingestion-topic") \
     .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [ ]:
# Create a small temporary view for SparkSQL
df.createOrReplaceTempView("message")

In [ ]:
# Write Out the message to the console of the environment
res = spark.sql("SELECT * FROM message")
res.writeStream.format("console") \
                .outputMode("append") \
                .start()

In [ ]:
# Write the message back into Kafka in another topic that you are going to listen to with a local consumer
ds = df \ 
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "spark-output") \
    .option("checkpointLocation", "/tmp") \
    .start() \
    .awaitTermination()